In [1]:
from gensim.corpora import Dictionary, MmCorpus
import numpy as np
from tqdm import tqdm_notebook as tqdm
from tables import *

In [2]:
data_path = "/mnt/disks/sdb/data/pub_full.h5"
dict_path = "/mnt/disks/sdb/data/dct.p"
cits_path = "/mnt/disks/sdb/data/DictCits.h5"
out_path = "/mnt/disks/sdb/data/"

In [5]:
dict_size = 1000000

In [10]:
h5file_data = open_file(data_path, mode="r", title="pub_sample")
table_data = h5file_data.root.pub_exp.sample

In [3]:
dictionary = Dictionary.load(dict_path)

In [5]:
len(dictionary)

2000478

In [14]:
nb_l_data = len(table_data)

In [46]:
DictAbs = {}
for i in tqdm(range(nb_l_data)):
    line = table_data[i]
    Art_Id = line['Id_Art']
    if Art_Id in DictAbs.keys():
        DictAbs[Art_Id].append(i)
    else:
        DictAbs[Art_Id] = [i]
abs_cits = DictAbs.keys()

/home/User1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


A Jupyter Widget

In [57]:
import pickle


In [61]:
with open(out_path + 'abs_cits.txt','w') as f:
    for key in list(abs_cits):
        f.write(str(key) + '\n')

In [49]:
def read_ID(idx,table):
    rows = table[idx]['Abstract']
    rows = [r.decode("utf-8") for r in rows]
    return ' '.join(rows).split(' ')
class MyCorpus(object):
    def __iter__(self):
        for Id_Art in abs_cits:
            # assume there's one document per line, tokens separated by whitespace
            abstract = read_ID(DictAbs[Id_Art],table_data)
            yield dictionary.doc2bow(abstract)

In [ ]:
MmCorpus.serialize(out_path+'pub_full.mm', MyCorpus())

In [52]:
mm = MmCorpus(out_path + 'pub_full.mm')

In [53]:
from gensim.models import TfidfModel

In [54]:
tfidf = TfidfModel(mm, id2word=dictionary)

In [55]:
tfidf.save(out_path +  'pub_full.tfidf_model')

In [56]:
MmCorpus.serialize(out_path + 'pub_full_tfidf.mm', tfidf[mm], progress_cnt=10000)